In [19]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score
import torch.utils.data as Data
from transformers import Trainer, TrainingArguments

In [2]:
emotions = load_dataset("dataset")
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
model_ckpt = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [5]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)


# 批次处理,整个数据集同时进行处理
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded  # 原有数据与map函数新增数据('input_ids', 'token_type_ids', 'attention_mask')的联合

Loading cached processed dataset at dataset/train/cache-9aa6937aa7a0509c.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at dataset/test/cache-b0d3f5da2aa8927a.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [6]:
emotions_encoded = emotions_encoded.remove_columns(['text'])  # 'text'列不参与训练(即不进入自定义模型forward函数)
emotions_encoded.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])
emotions_encoded['train'][:]  # 字典形式

{'label': tensor([0, 0, 3,  ..., 1, 3, 0]),
 'input_ids': tensor([[  101,  1045,  2134,  ...,     0,     0,     0],
         [  101,  1045,  2064,  ...,     0,     0,     0],
         [  101, 10047,  9775,  ...,     0,     0,     0],
         ...,
         [  101,  1045,  2514,  ...,     0,     0,     0],
         [  101,  1045,  2514,  ...,     0,     0,     0],
         [  101,  1045,  2113,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [7]:
class Customize_Model(torch.nn.Module):
    """下游训练任务模型"""

    def __init__(self, pretrained_model, num_labels):
        super().__init__()
        self.classifier = torch.nn.Linear(768, num_labels)  # 多分类任务
        self.pretrained = pretrained_model
        self.dropout = nn.Dropout()
        self.loss_fct = nn.CrossEntropyLoss()  # 损失函数

    def forward(self,
                input_ids,  # ★★★★★训练阶段对应emotions_encoded['train']中的input_ids
                attention_mask,  # ★★★★★训练阶段对应emotions_encoded['train']中的attention_mask
                token_type_ids,  # ★★★★★训练阶段对应emotions_encoded['train']中的token_type_ids
                labels=None):  # 标签;★★★★★训练阶段对应emotions_encoded['train']中的labels
        outputs = self.pretrained(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  token_type_ids=token_type_ids)
        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:  # 若包含标签
            loss = self.loss_fct(logits.view(-1, num_labels), labels.view(-1))

        # ★★★★★
        # 返回值为一个元组
        # 元组的第一个元素必须为该批次训练数据的损失值
        # 元素的第二个元素用于评估函数计算验证数据集(若有)的输出
        return (loss, logits)


num_labels = 6
model_from_pretrained = AutoModel.from_pretrained(model_ckpt)

model = Customize_Model(model_from_pretrained, num_labels)
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def compute_metrics(pred):
    """验证数据集评估函数"""
    labels = pred.label_ids  # 对应自定义模型forward函数输入:labels
    preds = pred.predictions  # 对应自定义模型forward函数返回值的第二个元素
    preds_argmax = preds.argmax(-1)
    f1 = f1_score(labels, preds_argmax, average="weighted")
    acc = accuracy_score(labels, preds_argmax)
    return {"accuracy": acc, "f1": f1}  # return a dictionary string to metric values

In [9]:
# 主要调节的超参数
batch_size = 64
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written.
    output_dir=model_name,
    seed=42,

    # Total number of training epochs to perform
    num_train_epochs=2,  # 默认:3.0
    # If set to a positive number, the total number of training steps to perform. Overrides num_train_epochs. I
    # max_steps=2000,  # 默认:-1

    #  Maximum gradient norm (for gradient clipping).
    max_grad_norm=1.0,  # 默认:1.0

    # The batch size per GPU/TPU core/CPU for training.
    per_device_train_batch_size=batch_size,  # 默认:8
    # The batch size per GPU/TPU core/CPU for evaluation.
    per_device_eval_batch_size=batch_size,  # 默认:8

    # HuggingFace AdamW优化器超参数
    # The initial learning rate for AdamW optimizer.
    learning_rate=2e-5,  # 默认: 5e-5
    # The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
    weight_decay=0.01,  # 默认:0
    # The beta1 hyperparameter for the AdamW optimizer.
    adam_beta1=0.9,  # 默认:0.9
    # The beta2 hyperparameter for the AdamW optimizer.
    adam_beta2=0.999,  # 默认:0.999
    # The epsilon hyperparameter for the AdamW optimizer.
    adam_epsilon=1e-8,  # 默认:1e-8

    # HuggingFace 不同学习率预热超参数
    #  The scheduler type to use. See the documentation of SchedulerType for all possible values.
    lr_scheduler_type='linear',  # 默认:'linear'
    # Ratio of total training steps used for a linear warmup from 0 to learning_rate.
    warmup_ratio=0.0,  # 默认:0.0
    # Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
    warmup_steps=0,  # 默认0

    # The evaluation strategy to adopt during training. Possible values are:
    # "no": No evaluation is done during training.
    # "steps": Evaluation is done (and logged) every eval_steps.
    # "epoch": Evaluation is done at the end of each epoch.
    evaluation_strategy="epoch",  # 默认:'no'
    # The logging strategy to adopt during training. Possible values are:
    # "no": No logging is done during training.
    # "epoch": Logging is done at the end of each epoch.
    # "steps": Logging is done every logging_steps.
    logging_strategy='epoch',  # 默认:'steps'
    # Number of update steps between two logs if logging_strategy="steps".
    # logging_steps=500,  # 默认:500
    # The checkpoint save strategy to adopt during training. Possible values are:
    # "no": No save is done during training.
    # "epoch": Save is done at the end of each epoch.
    # "steps": Save is done every save_steps.
    # Logger log level to use on the main process. Possible choices are the log levels as strings: ‘debug’, ‘info’, ‘warning’, ‘error’ and ‘critical’, plus a ‘passive’ level which doesn’t set anything and lets the application set the level.
    log_level='passive',  # 默认'passive'
    save_strategy='epoch',  # 默认:'steps'
    # Number of updates steps before two checkpoint saves if save_strategy="steps".
    # save_steps=500,  # 默认:500
    disable_tqdm=False,  # 使用tqdm显示进度
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=emotions_encoded["train"],  # 类型:datasets.arrow_dataset.Dataset
    eval_dataset=emotions_encoded["validation"],  # 类型:datasets.arrow_dataset.Dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)
trainer.train()  # 模型训练

/root/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 500


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.944800,0.323132,0.901000,0.898926
2,0.253600,0.214852,0.920500,0.920510


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to bert-base-uncased-finetuned-emotion/checkpoint-250
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in bert-base-uncased-finetuned-emotion/checkpoint-250/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-emotion/checkpoint-250/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to bert-base-uncased-finetuned-emotion/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in bert-base-uncased-finetuned-emotion/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-emotion/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=500, training_loss=0.5991856842041016, metrics={'train_runtime': 118.4985, 'train_samples_per_second': 270.046, 'train_steps_per_second': 4.219, 'total_flos': 0.0, 'train_loss': 0.5991856842041016, 'epoch': 2.0})

In [10]:
# Run prediction and returns predictions and potential metrics.
preds_output = trainer.predict(emotions_encoded["validation"])  # 类型:datasets.arrow_dataset.Dataset;包含标签
preds_output

***** Running Prediction *****
  Num examples = 2000
  Batch size = 64


PredictionOutput(predictions=array([[ 4.7823853 , -0.47211242, -0.8000455 , -0.8192299 , -1.4066333 ,
        -1.8550745 ],
       [ 4.7416277 , -0.36532468, -1.547922  , -0.49605307, -0.76526946,
        -1.6814713 ],
       [-1.4562929 ,  1.9604354 ,  3.1431172 , -1.2758981 , -1.7712132 ,
        -1.4010117 ],
       ...,
       [-1.3463206 ,  4.849283  ,  0.2021552 , -2.022539  , -2.0852394 ,
        -0.6736212 ],
       [-2.4475095 ,  3.0625157 ,  2.5931797 , -1.4979831 , -1.6729348 ,
        -1.100731  ],
       [-1.7385969 ,  4.770994  ,  0.20426473, -2.086232  , -1.8465284 ,
        -0.14392798]], dtype=float32), label_ids=array([0, 0, 2, ..., 1, 1, 1]), metrics={'test_loss': 0.2148517519235611, 'test_accuracy': 0.9205, 'test_f1': 0.9205104721800366, 'test_runtime': 1.9886, 'test_samples_per_second': 1005.755, 'test_steps_per_second': 16.092})

In [11]:
print(preds_output.predictions)  # 预测结果
print(type(preds_output.predictions))
print(preds_output.predictions.shape)

[[ 4.7823853  -0.47211242 -0.8000455  -0.8192299  -1.4066333  -1.8550745 ]
 [ 4.7416277  -0.36532468 -1.547922   -0.49605307 -0.76526946 -1.6814713 ]
 [-1.4562929   1.9604354   3.1431172  -1.2758981  -1.7712132  -1.4010117 ]
 ...
 [-1.3463206   4.849283    0.2021552  -2.022539   -2.0852394  -0.6736212 ]
 [-2.4475095   3.0625157   2.5931797  -1.4979831  -1.6729348  -1.100731  ]
 [-1.7385969   4.770994    0.20426473 -2.086232   -1.8465284  -0.14392798]]
<class 'numpy.ndarray'>
(2000, 6)


In [12]:
preds_output.metrics

{'test_loss': 0.2148517519235611,
 'test_accuracy': 0.9205,
 'test_f1': 0.9205104721800366,
 'test_runtime': 1.9886,
 'test_samples_per_second': 1005.755,
 'test_steps_per_second': 16.092}

In [13]:
test_dataset = emotions_encoded["test"].remove_columns(['label'])
test_dataset  # 测试数据集不含标签

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [20]:
test_torch_dataset = Data.TensorDataset(test_dataset['input_ids'], test_dataset['token_type_ids'],
                                        test_dataset['attention_mask'])
test_torch_dataset  # 数据加工

In [25]:
dataLoader_test = Data.DataLoader(dataset=test_torch_dataset, batch_size=2)  # 数据处理
for i in dataLoader_test:
    print(i[0])
    print(i[1])
    print(i[2])
    break

tensor([[  101, 10047,  3110,  2738, 11083,  2061, 10047,  2025,  2200, 12479,
          2157,  2085,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101, 10047,  2039, 16616,  2026,  9927,  2138,  1045,  2514, 28543,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,


In [26]:
def predict(model, data_loader):
    """预测不含标签的测试数据集"""
    model.eval()  # Sets the module in evaluation mode.
    predict_list = []
    with torch.no_grad():
        for i in data_loader:
            input_ids = i[0].to(device)
            attention_mask = i[1].to(device)
            token_type_ids = i[2].to(device)
            output = model(input_ids, attention_mask, token_type_ids)[1]
            predict_list.append(output)
    predict_all = torch.cat(predict_list, dim=0)
    return predict_all


result = predict(model, dataLoader_test)
print(result)

tensor([[ 3.4552, -0.3450, -1.0418,  0.3081, -0.6800, -1.3975],
        [ 3.6766, -0.7781, -0.8919,  0.0936, -0.5768, -1.5109],
        [ 2.9260, -0.9456, -0.9254,  0.4611,  0.3105, -1.0101],
        ...,
        [-0.3021,  1.3146,  0.0343,  0.0378,  0.2261, -0.0418],
        [-0.7132,  2.3728, -0.1887, -0.7444, -0.0694, -0.1836],
        [-1.0886, -1.4676, -0.5053, -1.1809,  2.9800,  2.1502]],
       device='cuda:0')
